# Attention Mechanism

## A Simple Self-Attention without Trainable Weights

In [1]:
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your
        [0.55, 0.87, 0.66],  # journey
        [0.57, 0.85, 0.64],  # starts
        [0.22, 0.58, 0.33],  # with
        [0.77, 0.25, 0.10],  # one
        [0.05, 0.80, 0.55]   # step
    ]
)

In [2]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
# Normalize attention_scores
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(f"Attention Weights: {attn_weights_2_tmp}")
print(f"Sum: {attn_weights_2_tmp.sum()}")

Attention Weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: 1.0000001192092896


In [4]:
# Softmax for normalizatiob
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(f"Attention weights: {attn_weights_2_naive}")
print(f"Sum: {attn_weights_2_naive.sum()}")

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


In [5]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(f"Attention weights: {attn_weights_2}")
print(f"Sum: {attn_weights_2.sum()}")

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


In [6]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [7]:
# Calculating attention weights for all Input tokens
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [8]:
# Using Matrix Multiplication instead of for loop
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [9]:
# Normalize
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [10]:
# Verify
row_2_sum = sum(attn_weights[1][:])
print(f"Row 2 sum: {row_2_sum}")

Row 2 sum: 1.0


In [11]:
# Compute all context vectors
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self-Attention with Trainable Weights

In [12]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [13]:
# Initialize the three weight matrices W_q, W_k, W_v
torch.manual_seed(314159)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [14]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.5779, 1.3467])


In [15]:
keys = inputs @ W_key
values = inputs @ W_value
print(f"keys.shape: {keys.shape}")
print(f"values.shape: {values.shape}")

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [16]:
# Compute Attention Scores
keys_2 = keys[1]
attn_scores_22 = query_2.dot(keys_2)
print(attn_scores_22)

tensor(1.5010)


In [17]:
attn_scores_2 = query_2 @keys.T
print(attn_scores_2)

tensor([1.2540, 1.5010, 1.4686, 0.8360, 0.4714, 1.2046])


In [18]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1773, 0.2112, 0.2064, 0.1319, 0.1020, 0.1712])


In [19]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.7261, 0.8764])


In [20]:
# Using Simple Attention mechanism
from attention import SelfAttention_v1

torch.manual_seed(314159)
sa_v1 = SelfAttention_v1(d_in=d_in, d_out=d_out)
print(sa_v1(inputs))

tensor([[0.7216, 0.8740],
        [0.7261, 0.8764],
        [0.7259, 0.8762],
        [0.7013, 0.8581],
        [0.7083, 0.8631],
        [0.7056, 0.8611]], grad_fn=<MmBackward0>)


In [21]:
# Using Simple Attention mechanism
from attention import SelfAttention_v2

torch.manual_seed(314159)
sa_v2 = SelfAttention_v2(d_in=d_in, d_out=d_out)
print(sa_v2(inputs))

tensor([[-0.1866,  0.1720],
        [-0.1893,  0.1666],
        [-0.1893,  0.1666],
        [-0.1900,  0.1656],
        [-0.1904,  0.1676],
        [-0.1895,  0.1652]], grad_fn=<MmBackward0>)


## Causal Attention

In [22]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1525, 0.1755, 0.1751, 0.1659, 0.1602, 0.1708],
        [0.1658, 0.1673, 0.1671, 0.1671, 0.1640, 0.1687],
        [0.1658, 0.1673, 0.1671, 0.1671, 0.1642, 0.1685],
        [0.1685, 0.1656, 0.1656, 0.1672, 0.1655, 0.1676],
        [0.1657, 0.1672, 0.1673, 0.1662, 0.1682, 0.1655],
        [0.1685, 0.1657, 0.1655, 0.1676, 0.1636, 0.1691]],
       grad_fn=<SoftmaxBackward0>)


In [23]:
# Create a mask
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [24]:
# Apply the mask for the attention weights
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1525, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1658, 0.1673, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1658, 0.1673, 0.1671, 0.0000, 0.0000, 0.0000],
        [0.1685, 0.1656, 0.1656, 0.1672, 0.0000, 0.0000],
        [0.1657, 0.1672, 0.1673, 0.1662, 0.1682, 0.0000],
        [0.1685, 0.1657, 0.1655, 0.1676, 0.1636, 0.1691]],
       grad_fn=<MulBackward0>)


In [25]:
# Renormalize the attention weights
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4978, 0.5022, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3315, 0.3344, 0.3341, 0.0000, 0.0000, 0.0000],
        [0.2527, 0.2484, 0.2483, 0.2507, 0.0000, 0.0000],
        [0.1985, 0.2003, 0.2004, 0.1992, 0.2016, 0.0000],
        [0.1685, 0.1657, 0.1655, 0.1676, 0.1636, 0.1691]],
       grad_fn=<DivBackward0>)


In [26]:
# A more effective masking
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[ 0.0287,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0007,  0.0131,    -inf,    -inf,    -inf,    -inf],
        [ 0.0008,  0.0134,  0.0122,    -inf,    -inf,    -inf],
        [-0.0045, -0.0288, -0.0294, -0.0159,    -inf,    -inf],
        [ 0.0028,  0.0154,  0.0161,  0.0072,  0.0243,    -inf],
        [-0.0054, -0.0294, -0.0308, -0.0135, -0.0478, -0.0009]],
       grad_fn=<MaskedFillBackward0>)


In [27]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4978, 0.5022, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3315, 0.3344, 0.3341, 0.0000, 0.0000, 0.0000],
        [0.2527, 0.2484, 0.2483, 0.2507, 0.0000, 0.0000],
        [0.1985, 0.2003, 0.2004, 0.1992, 0.2016, 0.0000],
        [0.1685, 0.1657, 0.1655, 0.1676, 0.1636, 0.1691]],
       grad_fn=<SoftmaxBackward0>)


### Masking additional attention weights with dropout

In [28]:
torch.manual_seed(314159)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))

tensor([[0., 0., 0., 2., 2., 0.],
        [2., 2., 2., 0., 2., 2.],
        [2., 0., 2., 0., 2., 0.],
        [2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2.],
        [0., 0., 2., 2., 2., 0.]])


In [29]:
# Apply dropout to attention weights
torch.manual_seed(314159)
print(dropout(attn_weights))

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9956, 1.0044, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6629, 0.0000, 0.6683, 0.0000, 0.0000, 0.0000],
        [0.5054, 0.4967, 0.4966, 0.5013, 0.0000, 0.0000],
        [0.3971, 0.4006, 0.4008, 0.3983, 0.4032, 0.0000],
        [0.0000, 0.0000, 0.3311, 0.3351, 0.3271, 0.0000]],
       grad_fn=<MulBackward0>)


In [30]:
# Create batch
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [31]:
# Verify Causal Attention implementation
from attention import CausalAttention

torch.manual_seed(314159)
context_length = batch.shape[1]

ca = CausalAttention(d_in=d_in, d_out=d_out, context_length=context_length, dropout=0.0)
context_vecs = ca(batch)
print(f"context_vecs.shape: {context_vecs.shape}")

context_vecs.shape: torch.Size([2, 6, 2])


## Extending Single-Head Attention to Multi-Head Attention

In [32]:
# Verify the use of a list of Causal Attentions as Multi-head attention
# In this case the Causal Attention will be processed sequentially
from attention import MultiHeadAttentionWrapper

torch.manual_seed(314159)
context_length = batch.shape[1]
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in=d_in, d_out=d_out, context_length=context_length, dropout=0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print(f"context_vecs.shape: {context_vecs.shape}")

tensor([[[-0.3760, -0.1756, -0.3800,  0.1722],
         [-0.2934,  0.0484, -0.3221,  0.1930],
         [-0.2686,  0.1255, -0.3025,  0.1965],
         [-0.2149,  0.1355, -0.2565,  0.1813],
         [-0.2300,  0.1815, -0.2340,  0.1389],
         [-0.1895,  0.1652, -0.2215,  0.1593]],

        [[-0.3760, -0.1756, -0.3800,  0.1722],
         [-0.2934,  0.0484, -0.3221,  0.1930],
         [-0.2686,  0.1255, -0.3025,  0.1965],
         [-0.2149,  0.1355, -0.2565,  0.1813],
         [-0.2300,  0.1815, -0.2340,  0.1389],
         [-0.1895,  0.1652, -0.2215,  0.1593]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [33]:
# Verify MultiHeadAttention for parallel instead of sequential
from attention import MultiHeadAttention

torch.manual_seed(314159)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in=d_in, d_out=d_out, context_length=context_length, dropout=0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print(f"context_vecs.shape: {context_vecs.shape}")

tensor([[[-0.6960, -0.2379],
         [-0.7365, -0.2963],
         [-0.7514, -0.3150],
         [-0.7396, -0.3368],
         [-0.7563, -0.3366],
         [-0.7409, -0.3497]],

        [[-0.6960, -0.2379],
         [-0.7365, -0.2963],
         [-0.7514, -0.3150],
         [-0.7396, -0.3368],
         [-0.7563, -0.3366],
         [-0.7409, -0.3497]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [34]:
# Initialize MultiHeadAttention for smallest GPT-2 model
block_size = 1024
d_in, d_out = 768, 768
num_heads = 12
mha = MultiHeadAttention(d_in=d_in, d_out=d_out, context_length=block_size, dropout=0.0, num_heads=num_heads)